In [29]:
import numpy as np 
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lillianhong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/lillianhong/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


reference: https://stackabuse.com/text-classification-with-python-and-scikit-learn/

In [30]:
movie_data = load_files("txt_sentoken")
x, y = movie_data.data, movie_data.target 

In [31]:
documents = [] 

from nltk.stem import WordNetLemmatizer 

stemmer = WordNetLemmatizer()
print(len(x))

for sen in range(0, len(x)):
    doc = re.sub(r'\W', ' ', str(x[sen])) #remove whitespace
    doc = re.sub(r'\s+[a-zA-z]\s+', ' ', doc) #remove single chars
    doc = re.sub(r'^[a-zA-z]\s+', ' ', doc) #remove single chars from beginning
    doc = re.sub(r'\s+^[a-zA-z]', ' ', doc) #remove single chars from end 
    doc = re.sub(r'\s+', ' ', doc, flags=re.I) #replace all whitespaces with single whitespace
    doc = re.sub(r'^b\s+', '', doc) #remove b in front of strings for binary doc
    doc = doc.lower()
    
    #lemmatization - standardizes words 
    doc = doc.split() 
    doc = [stemmer.lemmatize(word) for word in doc]
    doc = ' '.join(doc) 
    
    documents.append(doc)

2000


In [33]:
#vectorize words 
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
x = vectorizer.fit_transform(documents).toarray() 

TFIDF = term frequency * (1/document frequency) 
Meant to account for common words in documents
Below is conversion to TFIDF values:

In [35]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
x = tfidfconverter.fit_transform(x).toarray()

In [36]:
#split dataset

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)


In [38]:
# train with RandomForestClassifier 
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [78]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier((500,), early_stopping = True)
classifier.fit(x_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(500,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [79]:
y_pred =classifier.predict(x_test)

In [80]:
comb = y_pred + y_test
newcomb = comb[comb != 1]
print(len(newcomb)/len(comb) * 100)

84.25
